This notebook is to cluster users in a given recommender dataset into groups using *k-means*.\
Currently the total number of clusters is manually defined through the variable k in block two.\
The final result is saved as .csv in the output dir

In [1]:
import os

# change dir for custom imports
os.chdir('../')

In [2]:
os.getcwd()

'C:\\Users\\clari\\Desktop\\M2 - Thesis\\Research\\Dr Jacques Bou Abdo\\Recommender System\\5 - Ensemble Learning Model\\Accuracy Metrics\\Group Validation'

In [3]:
#dataset_name = 'framework-ml-25m-subset' 

In [4]:
import pandas as pd
dataset_name = 'ml-25m-subset-nf2.2'
#dataset_name = 'framework-ml-25m-subset'  # Adjust this as needed

base_path = f"C:\\Users\\clari\\Desktop\\M2 - Thesis\\Research\\Dr Jacques Bou Abdo\\Recommender System\\5 - Ensemble Learning Model\\Accuracy Metrics\\Group Validation\\datasets\\{dataset_name}\\"

def load_data(dataset_name, base_path):
    ratings_path = f'{base_path}clean/ratings.csv'
    movies_path = f'{base_path}movies.csv'
    
    ratings = pd.read_csv(ratings_path)
    movies = pd.read_csv(movies_path)
    
    if 'nf' in dataset_name or '25m-subset' in dataset_name:
        ratings = ratings.rename(columns={'itemId': 'movieId'})
        all_ratings = pd.read_csv(r"C:\Users\clari\Desktop\M2 - Thesis\Research\Dr Jacques Bou Abdo\Recommender System\4 - Review\dataset\ml-25m\Full Data set (Used with NF3 First)\ratings.csv")
        
        ratings = pd.merge(ratings, movies, on='movieId').sort_values(by='userId')
        ratings_detailed = pd.merge(ratings, all_ratings, on=['userId', 'movieId', 'rating']).sort_values(by='userId').rename(columns={'genres_x': 'genres', 'timestamp_x': 'timestamp', 'title_x': 'title'})
        
    else:
        ratings = ratings.rename(columns={'itemId': 'movieId'})
        ratings_detailed = pd.merge(ratings, movies, on='movieId')
    
    return ratings_detailed

ratings_detailed = load_data(dataset_name, base_path)
print(f'Number of ratings: {len(ratings_detailed)}')

# Uncomment if saving is required
# ratings_detailed.to_csv(f'datasets/{dataset_name}/clean/ratings.csv', index=False)

ratings_detailed.head()

Number of ratings: 194882


,userId,movieId,rating,title,genres,user_group,isNoisy,title_y,genres_y,timestamp
0,107,3267,4.0,"Mariachi, El (1992)",Action|Crime|Thriller|Western,LEUG,0,"Mariachi, El (1992)",Action|Crime|Thriller|Western,1113517986
19,107,2966,3.5,"Straight Story, The (1999)",Adventure|Drama,LEUG,0,"Straight Story, The (1999)",Adventure|Drama,1116042370
18,107,5377,3.5,About a Boy (2002),Comedy|Drama|Romance,LEUG,0,About a Boy (2002),Comedy|Drama|Romance,1146426679
17,107,553,4.0,Tombstone (1993),Action|Drama|Western,LEUG,0,Tombstone (1993),Action|Drama|Western,1112450296
16,107,2599,3.5,Election (1999),Comedy,LEUG,0,Election (1999),Comedy,1112450353


In [5]:
ratings_detailed.head()

,userId,movieId,rating,title,genres,user_group,isNoisy,title_y,genres_y,timestamp
0,107,3267,4.0,"Mariachi, El (1992)",Action|Crime|Thriller|Western,LEUG,0,"Mariachi, El (1992)",Action|Crime|Thriller|Western,1113517986
19,107,2966,3.5,"Straight Story, The (1999)",Adventure|Drama,LEUG,0,"Straight Story, The (1999)",Adventure|Drama,1116042370
18,107,5377,3.5,About a Boy (2002),Comedy|Drama|Romance,LEUG,0,About a Boy (2002),Comedy|Drama|Romance,1146426679
17,107,553,4.0,Tombstone (1993),Action|Drama|Western,LEUG,0,Tombstone (1993),Action|Drama|Western,1112450296
16,107,2599,3.5,Election (1999),Comedy,LEUG,0,Election (1999),Comedy,1112450353


In [6]:
from clustering.helpers import transform_df
from clustering.kmeans import create_clsuters

# number of clusters
k = 100
ratings_detailed['user_cat'] = None
ratings_detailed['rating_group'] = None
ratings_detailed['item_cat'] = None
ratings_detailed['user_group'] = None
ratings_detailed['genres_x'] = None
ratings_detailed['timestamp_y'] = None
ratings_detailed['title_x'] = None
ratings_detailed['title_y'] = None
ratings_detailed['timestamp_x'] = None
ratings_detailed['genres_y'] = None
ratings_detailed['prediction'] = None
ratings_detailed['prediction_y'] = None
ratings_detailed['prediction_x'] = None



if 'nf1' in dataset_name:
        ratings_detailed = ratings_detailed.drop(columns=['user_cat', 'rating_group', 'item_cat', 'user_group', 'prediction','genres_y','genres_x','title_x','timestamp_x','title_y','timestamp_y','prediction_x','prediction_y'])
    #.rename(columns={'genres_x':'genres', 'title_x':'title','timestamp_x':'timestamp'})
 
elif (dataset_name == 'ml-5m-#2'):
#or ('#2' in dataset_name) or ('#3' in dataset_name) :
    ratings_detailed['isNoisy'] = ratings_detailed['isNoisy'].apply(lambda x: x.strip('[]'))
    ratings_detailed = ratings_detailed.drop(columns=['user_cat', 'rating_group', 'item_cat', 'user_group', 'prediction','genres_y','genres_x','title_x','timestamp_x','title_y','timestamp_y','prediction_x','prediction_y'])

    #ratings_detailed['isNoisy'] = ratings_detailed['isNoisy'].strip('[]').astype(int).copy()  
elif dataset_name == 'ml-5m':
         ratings_detailed = ratings_detailed.drop(columns=['user_cat', 'rating_group', 'item_cat', 'user_group', 'prediction','genres_y','genres_x','title_x','timestamp_x','title_y','timestamp_y','prediction_x','prediction_y'])

else:
     ratings_detailed['prediction'] = None
     ratings_detailed = ratings_detailed.drop(columns=['user_cat', 'rating_group', 'item_cat', 'user_group', 'prediction','genres_y','genres_x','title_x','timestamp_x','title_y','timestamp_y','prediction_x','prediction_y'])

if '#2' in dataset_name:
    ratings_detailed = ratings_detailed.drop(columns=['isNoisy[]'])
#ratings_detailed = ratings_detailed.copy().rename()
ratings_detailed.head()

,userId,movieId,rating,title,genres,isNoisy,timestamp
0,107,3267,4.0,"Mariachi, El (1992)",Action|Crime|Thriller|Western,0,1113517986
19,107,2966,3.5,"Straight Story, The (1999)",Adventure|Drama,0,1116042370
18,107,5377,3.5,About a Boy (2002),Comedy|Drama|Romance,0,1146426679
17,107,553,4.0,Tombstone (1993),Action|Drama|Western,0,1112450296
16,107,2599,3.5,Election (1999),Comedy,0,1112450353


In [7]:
# use k means to cluster the users in the dataset

transformed_df = transform_df(ratings_detailed)
ratings_clustered = create_clsuters(k, transformed_df)
ratings_clustered

C:\Users\clari\Desktop\M2 - Thesis\Research\Dr Jacques Bou Abdo\Recommender System\5 - Ensemble Learning Model\Accuracy Metrics\Group Validation\clustering\helpers.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dataset.loc[dataset['genres'].str.contains(i), i] = 1


,userId,movieId,rating,timestamp,cluster
0,107,3267,4.0,1113517986,63
19,107,2966,3.5,1116042370,86
18,107,5377,3.5,1146426679,38
17,107,553,4.0,1112450296,63
16,107,2599,3.5,1112450353,63
...,...,...,...,...,...
194871,162488,7373,4.5,1202824760,36
194870,162488,4874,4.0,1202824779,36
194880,162488,2329,4.0,1202822271,36
194874,162488,745,1.0,1202822249,36


In [8]:
ratings_clustered.to_csv('output/' + dataset_name + '/clusters.csv', index=False)

# Load clusters

In [9]:
import pandas as pd

clusters = pd.read_csv('output/' + dataset_name + '/clusters.csv')

In [10]:
cluster_id = 20

# unique users in a certain cluster c
x = list(set(clusters[clusters['cluster'] == cluster_id].userId.to_list()))
x

[67063,
 91146,
 57356,
 94735,
 44048,
 68114,
 111123,
 46101,
 64535,
 70169,
 5659,
 81449,
 18989,
 42029,
 100909,
 81969,
 133681,
 83514,
 161342,
 14400,
 138817,
 74309,
 141385,
 159822,
 123473,
 22610,
 36855,
 89683,
 67157,
 125014,
 67673,
 98911,
 101471,
 106084,
 34924,
 112239,
 67700,
 11896,
 84600,
 109176,
 111228,
 157822,
 21632,
 29830,
 70279,
 63112,
 109702,
 145034,
 148109,
 3218,
 95380,
 77974,
 17047,
 156310,
 149145,
 47258,
 94882,
 5283,
 104612,
 110243,
 140454,
 16552,
 63144,
 132777,
 20653,
 21168,
 114354,
 101556,
 17080,
 39096,
 138424,
 150713,
 36028,
 90812,
 57027,
 60101,
 125638,
 48839,
 45768,
 72905,
 122059,
 30417,
 8183,
 65236,
 138455,
 77021,
 98013,
 160994,
 144611,
 100078,
 93424,
 76531,
 121589,
 61686,
 14583,
 70393,
 69883,
 18175,
 25857,
 8964,
 6917,
 66820,
 97543,
 139531,
 78092,
 22799,
 84752,
 129817,
 113440,
 75558,
 69424,
 318,
 75582,
 119104,
 44354,
 104274,
 124244,
 1367,
 84824,
 69977,
 96091,


In [11]:
clusters[clusters["cluster"] == 20]

,userId,movieId,rating,timestamp,cluster
102,318,46976,4.0,1186515097,20
126,318,51088,3.5,1186514630,20
128,318,50064,3.0,1186514760,20
134,318,46972,4.0,1186514765,20
137,318,51903,3.5,1186514575,20
...,...,...,...,...,...
193093,160994,41566,4.5,1185068170,20
193094,160994,8798,4.5,1185209131,20
193307,161157,5377,3.0,1187968441,20
193494,161342,48319,3.5,1186697898,20
